In [92]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName("MiAplicacion") \
    .getOrCreate()

In [104]:
df = spark.read.parquet("data2/final/merge_jugadores.parquet")
df.show(5)

+----------------+-------------+---------+-----------+------+---+------+----+---+---+-------+----+----+----+----+---+---+-------+-----+----------+------------+------------+------------+----+---+----+---+------+-----------+----------+----------+---+--------+---------+---------------+----------------+--------------+---+---+-----+----------------+---+-----+-------------+-------------+-------------+-------------+--------+----------+-------+---------------+---------------+--------------------+-------------------+---+---+----+---+----+-------+----+----------------+------+----------+
|          Player|        Squad|   Season|Competition|  Born| MP|Starts| Min|Gls|Ast|YellowC|RedC|PrgC|PrgP|PrgR|Fls|Off|Crosses|Recov|Aerialwon%|Tack_Def_3rd|Tack_Mid_3rd|Tack_Att_3rd|Tkl%|Tkl|TklW|Int|Blocks|Block_Shots|Bolck_Pass|Clearences|Err|Pass_cmp|Pass_cmp%|Pass_cmp_Short%|Pass_cmp_Medium%|Pass_cmp_Long%|xAG| xA|A-xAG|Pass_cmp_Att_3rd|PPA|CrsPA|Touch_Def_3rd|Touch_Mid_3rd|Touch_Att_3rd|Touch_Att_Pen|drib

In [105]:
df_unidos = df.toPandas()
df_unidos.to_csv("data2/unidos.csv", index=False)

df.columns

In [98]:
df.columns

['Player',
 'Squad',
 'Season',
 'Competition',
 'Born',
 'MP',
 'Starts',
 'Min',
 'Gls',
 'Ast',
 'YellowC',
 'RedC',
 'PrgC',
 'PrgP',
 'PrgR',
 'Fls',
 'Off',
 'Crosses',
 'Recov',
 'Aerialwon%',
 'Tack_Def_3rd',
 'Tack_Mid_3rd',
 'Tack_Att_3rd',
 'Tkl%',
 'Tkl',
 'TklW',
 'Int',
 'Blocks',
 'Block_Shots',
 'Bolck_Pass',
 'Clearences',
 'Err',
 'Pass_cmp',
 'Pass_cmp%',
 'Pass_cmp_Short%',
 'Pass_cmp_Medium%',
 'Pass_cmp_Long%',
 'xAG',
 'xA',
 'A-xAG',
 'Pass_cmp_Att_3rd',
 'PPA',
 'CrsPA',
 'Touch_Def_3rd',
 'Touch_Mid_3rd',
 'Touch_Att_3rd',
 'Touch_Att_Pen',
 'drib_Att',
 'drib_Succ%',
 'PrgDist',
 'Carries_Att_3rd',
 'Carries_Att_Pen',
 'fail_To_Gain_Control',
 'Loss_Control_Tackle',
 'Sh',
 'SoT',
 'G/Sh',
 'xG',
 'npxG',
 'npxG/Sh',
 'G-xG',
 'Pos',
 'Height',
 'Value']

In [45]:
import pandas as pd
col_d = pd.read_csv("data/fbref_defense/laliga_2022_2023_defense.csv")
col_d.columns

Index(['Unnamed: 0_level_0', 'Unnamed: 1_level_0', 'Unnamed: 2_level_0',
       'Unnamed: 3_level_0', 'Unnamed: 4_level_0', 'Unnamed: 5_level_0',
       'Unnamed: 6_level_0', 'Unnamed: 7_level_0', 'Tackles', 'Tackles.1',
       'Tackles.2', 'Tackles.3', 'Tackles.4', 'Challenges', 'Challenges.1',
       'Challenges.2', 'Challenges.3', 'Blocks', 'Blocks.1', 'Blocks.2',
       'Unnamed: 20_level_0', 'Unnamed: 21_level_0', 'Unnamed: 22_level_0',
       'Unnamed: 23_level_0', 'Unnamed: 24_level_0'],
      dtype='object')

In [25]:
df3 = spark.read.parquet("data/final/merge_jugadores.parquet")
df3.show(5)

+--------------------+---------+--------------+--------+------+----------------+-------------+---------------+-----+-----------+---------+-----+--------------------+-----------------+---------------------------+---------------+---------------------+---------------------+----------+-----------------+------------------+------------+------------+------------+--------------+--------+----------------+--------+-------------+-------------+-----------+------+---------+------------+--------------+----------------------------+
|              nombre|temporada|          liga|Posicion|nacido|partidos_jugados|titularidades|minutos_jugados|goles|asistencias|amarillas|rojas|carreras_progresivas|pases_progresivos|Pases_progresivos_recibidos|goles_esperados|asistencias_esperadas|goles_esperados_noPen|key_passes|          xGChain|         xGBuildup|entradas_1/3|entradas_2/3|entradas_3/3|duelos_ganados|entradas|entradas_ganadas|bloqueos|bloqueos_tiro|bloqueos_pase|club_actual|altura|    valor|xG_xA_por_90

In [39]:
df_unidos = df.toPandas()
df_unidos.to_csv("unidos3.csv", index=False)

In [34]:
from pyspark.sql.functions import col
df1= df.filter(col("valor").isNull())
df1_pd = df1.toPandas()
df1_pd.to_csv("nounidos4.csv", index=False)

In [2]:

import os
import pandas as pd
from pathlib import Path
import unicodedata
import re
from rapidfuzz import process
df_fbref = pd.read_csv("data/limpios/fbref/laliga_2022_2023.csv")
df_mercado = pd.read_csv("data/limpios/mercado/valores_mercado_LaLiga_2022-2023.csv")



In [15]:
def mapear_nombres(lista_fuente, lista_objetivo, umbral=85):
    mapping = {}
    for nombre in lista_fuente:
        mejor, score, _ = process.extractOne(nombre, lista_objetivo)
        if score >= umbral:
            mapping[nombre] = mejor
    return mapping

# Ejemplo con understat y fbref
nombres_mercado = df_mercado["nombre"].unique().tolist()
nombres_fbref = df_fbref["nombre"].unique().tolist()

mapping = mapear_nombres(nombres_mercado, nombres_fbref)

# Reemplazar en understat los nombres
df_mercado["nombre"] = df_mercado["nombre"].replace(mapping)

df_mercado["nombre"].to_csv("prueba_data/mercado_nombres.csv", index=False)

In [3]:
df_fbref = pd.read_csv("data/limpios/fbref/fbref.csv")
df_fbref_defense = pd.read_csv("data/limpios/fbref_defense/fbref_defense.csv")
df_understat = pd.read_csv("data/limpios/understat/understat.csv")
df_mercado = pd.read_csv("data/limpios/mercado/mercado.csv")

In [19]:
df_fbref = pd.read_csv("data/limpios/fbref/fbref.csv")
df_fbref_defense = pd.read_csv("data/limpios/fbref_defense/fbref_defense.csv")
df_understat = pd.read_csv("data/limpios/understat/understat.csv")
df_mercado = pd.read_csv("data/limpios/mercado/mercado.csv")

fbref = df_fbref["club_actual"]
understat = df_understat["club_actual"]
mercado = df_mercado["club_actual"]

nombre_fbref = fbref.dropna().sort_values().unique()
nombre_understat = understat.dropna().sort_values().unique()
nombre_mercado = mercado.dropna().sort_values().unique()

pd.Series(nombre_fbref).to_csv("prueba_data/club_actual_fbref.csv", index=False)
pd.Series(nombre_understat).to_csv("prueba_data/club_actual_understat.csv", index=False)
pd.Series(nombre_mercado).to_csv("prueba_data/club_actual_mercado.csv", index=False)

In [31]:
duplicados = df_fbref[df_fbref.duplicated(subset=["nombre"], keep=False)]
print(duplicados)
df_ordenado = duplicados.sort_values(by="nombre")
df_ordenado.to_csv("prueba_data/duplicados_fbref.csv", index=False)

                nombre Posicion  nacido  partidos_jugados  titularidades  \
0      paxten aaronson    MF,DF  2003.0                 7              0   
1       sargis adamyan    FW,MF  1993.0                24              4   
2        karim adeyemi    FW,MF  2002.0                24             20   
3           amine adli    FW,MF  2000.0                26             16   
4            felix agu    DF,FW  1999.0                 3              0   
...                ...      ...     ...               ...            ...   
6399   piotr zielinski       MF  1994.0                26              8   
6400      nadir zortea    DF,MF  1999.0                35             33   
6401  szymon zurkowski       MF  1997.0                 5              0   
6402        milan uric       FW  1990.0                18             13   
6403        milan uric       FW  1990.0                 9              3   

      minutos_jugados  goles  asistencias  amarillas  rojas  \
0                 173   

In [18]:
COLUMNAS_UTILES = {
    "fbref": [
        "Unnamed: 1_level_0_Player", "Unnamed: 4_level_0_Squad", "Unnamed: 3_level_0_Pos", "Unnamed: 6_level_0_Born", "Playing Time_MP", "Playing Time_Starts", "Playing Time_Min", "Performance_Gls", "Performance_Ast", "Performance_CrdY", "Performance_CrdR", "Progression_PrgC","Progression_PrgP","Progression_PrgR"
    ],
    "fbref_defense": [
       "Unnamed: 1_level_0_Player", "Unnamed: 4_level_0_Squad", "Tackles_Def 3rd", "Tackles_Mid 3rd","Tackles_Att 3rd", "Challenges_Tkl%", "Tackles_Tkl", "Tackles_TklW", "Int", "Blocks_Blocks", "Blocks_Sh", "Blocks_Pass", "Clr", "Err"
    ],
    "understat": [
        "nombre", "club_actual", "xG", "xA", "npxG", "shsots", "key_passes", "xGChain", "xGBuildup"
    ],
    "mercado": [
        "player_name", "current_club", "player_height_mtrs", "player_market_value_euro"
    ]
}

df_mercado = pd.read_csv("data/mercado/valores_mercado_LaLiga_2022-2023.csv")
mercado = ["player_name", "current_club", "player_height_mtrs", "player_market_value_euro"]
def limpiar_ligas_transfermarkt(df):
    ligas_principales = ["Serie A", "LaLiga", "Premier League", "Bundesliga"]
    return df[df["comp_name"].isin(ligas_principales)]

df_mercado = limpiar_ligas_transfermarkt(df_mercado)
columnas_utiles = COLUMNAS_UTILES.get("mercado", df_mercado.columns.tolist())
columnas_disponibles = [col for col in columnas_utiles if col in df_mercado.columns]
#df_mercado = df_mercado[mercado]
df_mercado.to_csv("prueba_data/mercadoLaLiga.csv", index=False)

In [40]:
df_fbref = pd.read_csv("data/limpios/fbref/fbref.csv")
df_fbref_defense = pd.read_csv("data/limpios/fbref_defense/fbref_defense.csv")
df_understat = pd.read_csv("data/limpios/understat/understat.csv")
df_mercado = pd.read_csv("data/limpios/mercado/mercado.csv")

col = ["nombre", "club_actual", "temporada", "liga"]

fbref= df_fbref[col].sort_values(by="nombre")
defense=df_fbref_defense[col].sort_values(by="nombre")
understat=df_understat[col].sort_values(by="nombre")
mercado=df_mercado[col].sort_values(by="nombre")

fbref.to_csv("prueba_data/limpios/fbref.csv", index=False)
defense.to_csv("prueba_data/limpios/defense.csv", index=False)
understat.to_csv("prueba_data/limpios/understat.csv", index=False)
mercado.to_csv("prueba_data/limpios/mercado.csv", index=False)


In [41]:
no_en_understat = fbref.merge(understat, on=["nombre", "club_actual", "temporada", "liga"], how="left", indicator=True)
no_en_understat = no_en_understat[no_en_understat["_merge"] == "left_only"]
print(no_en_understat)



               nombre club_actual  temporada            liga     _merge
4     aaron escandell  las palmas  2023-2024          laliga  left_only
19    abde ezzalzouli     osasuna  2022-2023          laliga  left_only
20    abde ezzalzouli       betis  2023-2024          laliga  left_only
21    abde ezzalzouli       betis  2024-2025          laliga  left_only
22    abde ezzalzouli   barcelona  2023-2024          laliga  left_only
...               ...         ...        ...             ...        ...
6332   yassine bounou     sevilla  2022-2023          laliga  left_only
6333   yassine bounou     sevilla  2023-2024          laliga  left_only
6346       yerry mina    cagliari  2023-2024         serie-a  left_only
6348  yerson mosquera      wolves  2024-2025  premier-league  left_only
6371    youssef maleh       lecce  2022-2023         serie-a  left_only

[594 rows x 5 columns]


In [48]:
import requests
from bs4 import BeautifulSoup

def obtener_links_stats(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "lxml")

    # Encuentra el bloque de la sección que contiene los enlaces
    sidebar = soup.find("div", {"id": "inner_nav"})

    if not sidebar:
        print("❌ No se encontró la barra lateral con los enlaces.")
        return

    # Extraer los enlaces (a tags) que contienen 'squad' o 'player' en el texto o URL
    enlaces = sidebar.find_all("a")

    print("📌 Enlaces de Squad & Player Stats:")
    for a in enlaces:
        texto = a.get_text(strip=True)
        href = a.get("href")
        if href and "comps/9" in href and any(keyword in href for keyword in [
            "standard", "keeper", "shooting", "passing", "possession", "defense", "misc"
        ]):
            print(f"- {texto}: https://fbref.com{href}")
url = "https://fbref.com/en/comps/9/2023-2024/stats/2023-2024-Premier-League-Stats"
obtener_links_stats(url)

📌 Enlaces de Squad & Player Stats:
- Goalkeeping: https://fbref.com/en/comps/9/2023-2024/keepers/2023-2024-Premier-League-Stats
- Advanced Goalkeeping: https://fbref.com/en/comps/9/2023-2024/keepersadv/2023-2024-Premier-League-Stats
- Shooting: https://fbref.com/en/comps/9/2023-2024/shooting/2023-2024-Premier-League-Stats
- Passing: https://fbref.com/en/comps/9/2023-2024/passing/2023-2024-Premier-League-Stats
- Pass Types: https://fbref.com/en/comps/9/2023-2024/passing_types/2023-2024-Premier-League-Stats
- Defensive Actions: https://fbref.com/en/comps/9/2023-2024/defense/2023-2024-Premier-League-Stats
- Possession: https://fbref.com/en/comps/9/2023-2024/possession/2023-2024-Premier-League-Stats
- Miscellaneous Stats: https://fbref.com/en/comps/9/2023-2024/misc/2023-2024-Premier-League-Stats
- Goalkeeping: https://fbref.com/en/comps/9/2023-2024/keepers/2023-2024-Premier-League-Stats
- Advanced Goalkeeping: https://fbref.com/en/comps/9/2023-2024/keepersadv/2023-2024-Premier-League-Stats

In [61]:
#fbref_scrappiing

import requests
from bs4 import BeautifulSoup, Comment
import pandas as pd
import os
from io import StringIO

ids_types = [("stats_standard", "stats"), ("stats_possession", "possession"), ("stats_defense", "defense"), ("stats_misc", "misc"), ("stats_passing", "passing"), ("stats_shooting", "shooting")]

def extraer_tabla_jugadores(url, table_id="stats_possession"):
    response = requests.get(url)
    if response.status_code != 200:
        raise ConnectionError(f"Error al acceder a {url}: código {response.status_code}")
    
    soup = BeautifulSoup(response.content, "lxml")
    comentarios = soup.find_all(string=lambda text: isinstance(text, Comment))

    for c in comentarios:
        if table_id in c:
            soup_comentario = BeautifulSoup(c, "lxml")
            tabla = soup_comentario.find("table", {"id": table_id})
            if tabla:
                df = pd.read_html(str(tabla))[0]
                if "Rk" in df.columns:
                    df = df[df["Rk"].apply(lambda x: str(x).isdigit())]
                return df

    raise ValueError(f"No se encontró la tabla {table_id} en {url}")

def build_defense_url(competition_id: int, season_label: str, type:str, slug:str) -> str:
    return f"https://fbref.com/en/comps/{competition_id}/{season_label}/{type}/{season_label}-{slug}-Stats"

def crear_tablas(competition_id: int, season_label: str, league_name: str):
    for id, type in ids_types:
        slug = league_name.replace(" ", "-")
        fbref_D_url = build_defense_url(competition_id, season_label, type, slug)
        df = extraer_tabla_jugadores(fbref_D_url, id)
        nombre_archivo = f"data2/{type}/{slug}_{season_label.replace('-', '_')}_{type}.csv"
        df.to_csv(nombre_archivo, index=False)
        print(f"✅ Guardado: {nombre_archivo}")

C:\Users\jahoy\AppData\Local\Temp\ipykernel_22708\3670577760.py:12: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(tabla))[0]


🧾 Columnas encontradas: [('Unnamed: 0_level_0', 'Rk'), ('Unnamed: 1_level_0', 'Player'), ('Unnamed: 2_level_0', 'Nation'), ('Unnamed: 3_level_0', 'Pos'), ('Unnamed: 4_level_0', 'Squad'), ('Unnamed: 5_level_0', 'Age'), ('Unnamed: 6_level_0', 'Born'), ('Unnamed: 7_level_0', '90s'), ('Touches', 'Touches'), ('Touches', 'Def Pen'), ('Touches', 'Def 3rd'), ('Touches', 'Mid 3rd'), ('Touches', 'Att 3rd'), ('Touches', 'Att Pen'), ('Touches', 'Live'), ('Take-Ons', 'Att'), ('Take-Ons', 'Succ'), ('Take-Ons', 'Succ%'), ('Take-Ons', 'Tkld'), ('Take-Ons', 'Tkld%'), ('Carries', 'Carries'), ('Carries', 'TotDist'), ('Carries', 'PrgDist'), ('Carries', 'PrgC'), ('Carries', '1/3'), ('Carries', 'CPA'), ('Carries', 'Mis'), ('Carries', 'Dis'), ('Receiving', 'Rec'), ('Receiving', 'PrgR'), ('Unnamed: 30_level_0', 'Matches')]
⚠️ Columna 'Rk' no encontrada. Devolviendo DataFrame completo.
  Unnamed: 0_level_0 Unnamed: 1_level_0 Unnamed: 2_level_0 Unnamed: 3_level_0  \
                  Rk             Player      

In [84]:
defense = pd.read_csv("data2/defense/La-Liga_2022_2023_defense.csv")
misc = pd.read_csv("data2/misc/La-Liga_2022_2023_misc.csv")
passing = pd.read_csv("data2/passing/La-Liga_2022_2023_passing.csv")
possession = pd.read_csv("data2/possession/La-Liga_2022_2023_possession.csv")
shooting = pd.read_csv("data2/shooting/La-Liga_2022_2023_shooting.csv")
stats = pd.read_csv("data2/stats/La-Liga_2024_2025_stats.csv")

def aplanar_columnas(df):
    # Aplana MultiIndex si existe
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = ['_'.join(filter(None, col)).strip() for col in df.columns.values]
    # Elimina fila duplicada si contiene 'Player' como valor
    if "Player" in df.columns and df.iloc[0]["Player"] == "Player":
        df = df[df["Player"] != "Player"]
    return df



In [96]:
defense.columns

Index(['Rk', 'Player', 'Nation', 'Pos', 'Squad', 'Age', 'Born', '90s', 'Tkl',
       'TklW', 'Def 3rd', 'Mid 3rd', 'Att 3rd', 'Tkl.1', 'Att', 'Tkl%', 'Lost',
       'Blocks', 'Sh', 'Pass', 'Int', 'Tkl+Int', 'Clr', 'Err', 'Matches'],
      dtype='object')

In [81]:
df4 = aplanar_columnas(misc)

df4.columns

Index(['Unnamed: 0_level_0', 'Unnamed: 1_level_0', 'Unnamed: 2_level_0',
       'Unnamed: 3_level_0', 'Unnamed: 4_level_0', 'Unnamed: 5_level_0',
       'Unnamed: 6_level_0', 'Unnamed: 7_level_0', 'Performance',
       'Performance.1', 'Performance.2', 'Performance.3', 'Performance.4',
       'Performance.5', 'Performance.6', 'Performance.7', 'Performance.8',
       'Performance.9', 'Performance.10', 'Performance.11', 'Performance.12',
       'Aerial Duels', 'Aerial Duels.1', 'Aerial Duels.2',
       'Unnamed: 24_level_0'],
      dtype='object')